In [3]:
from tensorflow.python.client import device_lib

import zipfile
import glob
import torch
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, concatenate, ConvLSTM2D, Conv3D, Input, MaxPooling3D
from tensorflow.keras import Model, layers, optimizers

import warnings
warnings.filterwarnings("ignore")

In [4]:
#GPU 사용 가능 여부 확인
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10831537334001737831
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3080192
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10572559301344059126
 physical_device_desc: "device: 0, name: NVIDIA A100-PCIE-40GB MIG 1g.5gb, pci bus id: 0000:1b:00.0, compute capability: 8.0"
 xla_global_id: 416903419]

In [5]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [11]:
def trainGenerator():
    for file in tr_file:
        dataset = np.load(file)
        target = dataset[:,:,-1].reshape(120,120,1)
        remove_minus= np.where(target<0,0,target)
        feature = dataset[:,:,:4]
        
    yield(feature, remove_minus)
    
def valGenerator():
    for file in val_file:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(120,120,1)        
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4]


        yield (feature, remove_minus)

In [6]:
def base_model(input_layer, start_neurons):
    convls1 = ConvLSTM2D(start_neurons*1, (3,3), activation = 'relu', padding = 'same', return_sequences=True)(input_layer)
    pool1 = BatchNormalization()(convls1)
    #pool1 = MaxPooling3D(2,2,2)(pool1)
    
    convls2 = ConvLSTM2D(start_neurons*2, (3,3), activation = 'relu', padding = 'same', return_sequences=True)(pool1)
    pool2 = BatchNormalization()(convls2)
    #pool2 = MaxPooling3D(2,2,2)(pool2)
    
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    pool1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(pool2)
    
    convlm = Conv2D(start_neurons*4, (3,3), activation = 'relu', padding = 'same')(pool2)
    
    deconv2 = Conv2DTranspose(start_neurons*2, (3,3), strides = (2,2), padding = 'same')(convlm)
    uconv2 = concatenate([deconv2, convls2])
    uconv2 = Conv2D(start_neurons*2, (3,3), activation = 'relu', padding='same')(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    
    deconv1 = Conv2DTranspose(start_neurons*1, (3,3), strides = (2,2), padding = 'same')(convlm)
    uconv1 = concatenate([deconv2, convls2])
    uconv1 = Conv2D(start_neurons*1, (3,3), activation = 'relu', padding='same')(uconv2)
    uconv1 = BatchNormalization()(uconv2)
    
    output_layer = Conv2D(1,(1,1),padding = 'same', activation = 'relu')(uconv1)
    
    return output_layer
    

In [ ]:
input_layer = Input((1,120,120,4))
output_layer = base_model(input_layer, 64)